# Food Delivery Data Integration & Analysis
This notebook performs the integration of three disparate data sources (CSV, JSON, and SQL) and conducts a series of analyses to answer business-critical questions.

## Step 1: Initial Set-up and Data Loading

In [ ]:
import pandas as pd
import json
import sqlite3
import os

# File paths
csv_path = 'orders.csv'
json_path = 'users.json'
sql_path = 'restaurants.sql'

### Load CSV (Transactional Data)

In [ ]:
orders_df = pd.read_csv(csv_path)
print(f'Orders loaded: {len(orders_df)} rows')
orders_df.head()

### Load JSON (User Master Data)

In [ ]:
users_df = pd.read_json(json_path)
print(f'Users loaded: {len(users_df)} rows')
users_df.head()

### Load SQL (Restaurant Master Data)
We will parse the SQL script and load it into a temporary SQLite database to extract the data into a DataFrame.

In [ ]:
with open(sql_path, 'r') as f:
    sql_script = f.read()

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.executescript(sql_script)
restaurants_df = pd.read_sql_query('SELECT * FROM restaurants', conn)
conn.close()
print(f'Restaurants loaded: {len(restaurants_df)} rows')
restaurants_df.head()

## Step 2: Data Merging
We perform left joins using `orders` as the base table to ensure all transactional data is preserved.

In [ ]:
# Join orders with users (Key: user_id)
merged_df = orders_df.merge(users_df, on='user_id', how='left')

# Join previous result with restaurants (Key: restaurant_id)
final_df = merged_df.merge(restaurants_df, on='restaurant_id', how='left', suffixes=('', '_master'))

# Save the final dataset
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print('Final dataset saved successfully.')
final_df.head()

## Step 3: Analytical Queries

### 1. City with highest revenue from Gold members

In [ ]:
gold_members = final_df[final_df['membership'] == 'Gold']
q1 = gold_members.groupby('city')['total_amount'].sum().idxmax()
print(f'Answer: {q1}')

### 2. Cuisine with highest average order value

In [ ]:
q2 = final_df.groupby('cuisine')['total_amount'].mean().idxmax()
print(f'Answer: {q2}')

### 3. Distinct users with total orders > INR 1000

In [ ]:
q3 = (final_df.groupby('user_id')['total_amount'].sum() > 1000).sum()
print(f'Answer: {q3}')

### 4. Rating range with highest revenue

In [ ]:
bins = [3.0, 3.55, 4.05, 4.55, 5.05]
labels = ['3.0 – 3.5', '3.6 – 4.0', '4.1 – 4.5', '4.6 – 5.0']
final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels, include_lowest=True)
q4 = final_df.groupby('rating_range', observed=True)['total_amount'].sum().idxmax()
print(f'Answer: {q4}')

### 5. Combination contributing highest revenue (Membership + Cuisine)

In [ ]:
target_combs = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]
q5_stats = final_df.groupby(['membership', 'cuisine'])['total_amount'].sum().reset_index()
q5_results = q5_stats[q5_stats.apply(lambda x: (x.membership, x.cuisine) in target_combs, axis=1)]
answer_q5 = q5_results.sort_values(by='total_amount', ascending=False).iloc[0]
print(f'Answer: {answer_q5.membership} + {answer_q5.cuisine}')

### 6. Percentage of orders by Gold members

In [ ]:
q6 = round((len(gold_members) / len(final_df)) * 100)
print(f'Answer: {q6}%')

### 7. Total revenue from Hyderabad (Rounded)

In [ ]:
q7 = round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum())
print(f'Answer: INR {q7}')

### 8. Quarter with highest total revenue

In [ ]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
q8 = final_df.groupby(final_df['order_date'].dt.to_period('Q'))['total_amount'].sum().idxmax()
print(f'Answer: {q8}')